* Tested with all cases from multiwaysplits(2) notebook; and scores are all correctly evaluated.

In [4]:
import pandas as pd
import random

def getcards(decks=1):
    """Brings out a number of sealed decks of cards in their original order to the virutal gaming table.

    Creates a dictionary tracking the name of each card, its quantity, and its value
    
    Keyword argument:
        decks {int} -- the number of 52-card decks that the casino is using (default 1)
    
    Returns:
        dictionary{keys: values} -- dictionary object where keys are the card name (e.g. 'K' for King )
                                    and where the values are a list object containing the card's quantity and its value.
                                    
    The value for the ace contains three elements, as aces can further take on the value of 1 or 11, depending on 
    the context of the hand. No distinction is made between card suits, i.e. spades, hearts, clubs, diamonds.
    """
    
    deck = {}
    total_cards = decks * 52
    card_quantity = int(total_cards / 13)
    
    deck['A'] = [card_quantity, 1, 11]

    for card in range(2, 11):
        deck[str(card)] = [card_quantity, card]
    
    for card in "JQK":
        deck[str(card)] = [card_quantity, 10]
        
    return deck, total_cards

deck, total_cards = getcards(decks=1)

def shuffler(deck, total_cards):
    
    """Shuffles the decks of cards, and places them inside the virtual gaming table card-shoe."""
    
    shuffled_shoe = []
    
    while len(shuffled_shoe) < total_cards:
        draw = random.choice(list(deck))
        if deck[draw][0] == 0:
            pass
        else:
            deck[draw][0] -= 1
            shuffled_shoe.append(draw)
            
    return shuffled_shoe

shuffled_shoe = shuffler(deck, total_cards)

In [13]:
# Work on computing player score here.
# Current thinking - when the player has a hard hand, computing player score is trivial.
# However, when player has a soft hand, it gets more tricky, and player score is multivalued. 
# It is multi-valued for more cases than the dealerscore - see the graphs you've drawn, and should include in write up
# Dealer score is multivalued only for a residual between 1 and 5, where residual is the sum of card values net of the 1st ace
# and for which subsequent aces have value 1. This is by virtue of the rules governing dealer behaviour for drawing
# cards. 

# Player score is multivalued for 1 <= residual <= 10. Due to this indeterminacy, it makes no sense to calculate the player's
# score, and it makes sense only to do it when the player has no outstanding decision-action sequences i.e. when his turn has 
# concluded. So we assume that the player's hand has terminated - this module will be called when 

# A heuristic I want to try is the following. Let's group by category of hand we will pass into the compute score module

# 1. Hard totals - trivial - sum of values. 
# 2. Splits - deal with this at the end.

# 3. Soft totals heuristic:

# Regardless of player hand,
# If residual >= 11, then hard total -> trivial computation
# If residual <= 10, then choose the ace to be the maximum value, so playerscore as close to 21 as possible.

In [5]:
# Module for evaluating hand, with some adaptation to arguments. Be aware that this was designed as part of dealerturn module.

def evaluate_hand(player):
    if 'A' in player:
        soft_total_indicator = 1
        print("Player has a soft total")
    else:
        soft_total_indicator = 0
        print("Player has a hard total")
    return soft_total_indicator

In [9]:
# Module for computing player score, given that all decision-action sequences have terminated. Tested in context of
# entire CF; and is working as expected. 

def compute_playerscore(player):
    soft_total_indicator = evaluate_hand(player)
    
    if soft_total_indicator:
        if player[0] != 'A':
            ace_index = player.index('A')
            player[0], player[ace_index] = player[ace_index], player[0]
            
        cardvalues_no_ace = [deck[card][1] for card in player[1::]]
        
        if sum(cardvalues_no_ace) >= 11:
            playerscore = deck['A'][1] + sum(cardvalues_no_ace)
        elif sum(cardvalues_no_ace) <= 10:
            playerscore = deck['A'][2] + sum(cardvalues_no_ace)   
    else:
        playerscore = sum([deck[card][1] for card in player])
        
    return playerscore

In [10]:
# Extension - to apply in the case of multiple hands.

def compute_multiple_playerscores(player_split_hands):
    playerscore_list = []
    for hand in player_split_hands:
        playerscore = compute_playerscore(hand)
        playerscore_list.append(playerscore)
    
    return playerscore_list

In [21]:
player_split_hands = [['A', '7'], ['A', '2'], ['A', '2'], ['A', '7'], ['A', '5'], ['A', '9'], ['A', '4'], ['A', '2'], ['A', '2'], ['A', '3'], ['A', '9']]

In [22]:
compute_multiple_playerscores(player_split_hands)

Player has a soft total
Player has a soft total
Player has a soft total
Player has a soft total
Player has a soft total
Player has a soft total
Player has a soft total
Player has a soft total
Player has a soft total
Player has a soft total
Player has a soft total


[18, 13, 13, 18, 16, 20, 15, 13, 13, 14, 20]